In [1]:
from __future__ import division

In [2]:
import os
import random
import math
import datetime
import time

In [3]:
import numpy as np
import pandas as pd

In [75]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers


# Train a new model
- Load the features/targets
- Create a model
- Save it with a nice name to 'models/{}'

In [136]:
# 100,000 rows of random vs random (gen 0) data
key = 'random-2017-10-21-13:41:47'

# 10,000 rows of gen-1 vs gen-1 data
key = 'gen-1-cov2d_beta_2017_10_22_142925'

# 5000 rows of covnet vs covnet (gen 1) data
#key = 'gen-1-2017-10-22-133109'

In [137]:
features = pd.read_csv('training_data/{}/features.csv'.format(key),
                       names=['game_idx', 'turn_idx'] + range(42),
                       dtype='int')

targets = pd.read_csv('training_data/{}/targets.csv'.format(key),
                      names=['game_idx', 'turn_idx', 'win', 'lose', 'draw'],
                      dtype='int')

In [138]:
# Split into training and holdout,
# ensuring separate games 

train_games = pd.Series(features.game_idx.unique()).sample(frac=0.8).values #games = #features.game_idx.unique()

features_train = features[features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
targets_train = targets[targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)

tidx = list(features_train.index)
random.shuffle(tidx)
features_train = features_train.loc[tidx]
targets_train = targets_train.loc[tidx]

features_test = features[~features.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)
targets_test = targets[~targets.game_idx.isin(train_games)].drop(['game_idx', 'turn_idx'], axis=1)

In [139]:
print len(features), len(features_train), len(features_test)
print len(targets), len(targets_train), len(targets_test)

print targets.win.value_counts(normalize=True)
print targets.lose.value_counts(normalize=True)
print targets.draw.value_counts(normalize=True)

145558 116499 29059
145558 116499 29059
1    0.518728
0    0.481272
Name: win, dtype: float64
0    0.522383
1    0.477617
Name: lose, dtype: float64
0    0.996345
1    0.003655
Name: draw, dtype: float64


In [98]:
def get_batch_iter(batch_size, batch_idx, dfs):
    
    length = len(dfs[0])
    for df in dfs:
        assert len(df) == length
        
    batches_per_df = int(math.ceil(length / batch_size))
    
    local_idx = batch_idx % batches_per_df
    
    start = local_idx*batch_size
    end = (local_idx+1)*batch_size
    
    return [df.iloc[start:end] for df in dfs]


def get_batch_random(batch_size, _, dfs):    
    mask = pd.Series(dfs[0].index).sample(n=batch_size, replace=False)    
    return [df.loc[mask] for df in dfs]

def get_batch(batch_size, batch_idx, dfs, how='iter'):
    if how == 'iter':
        return get_batch_iter(batch_size, batch_idx, dfs)
    elif how == 'random':
        return get_batch_random(batch_size, batch_idx, dfs)
    else:
        raise Exception()

In [99]:
def run(graph, output_prefix, batch_size, num_batches, batch_how='iter'):
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)
    
        #suffix = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
        #name = output_prefix #'{}_{}'.format(output_prefix, suffix)
        #save_name = "{}/model".format(name) #, suffix)

        train_writer = tf.summary.FileWriter(output_prefix, sess.graph)
        
        print "Running {}".format(output_prefix)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: features_test, outcome: targets_test}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: features_test, outcome: targets_test})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features_train, targets_train], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        model_dir = '{}/model'.format(output_prefix)
        print "SAVING TO: {}".format(model_dir)
        tf.train.Saver().save(sess, model_dir)

## Attempt 1: Normal dense fully-connected neural network

In [100]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy
from keras.regularizers import l2

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board')
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')

    # Fully connected layers
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(board)
    
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
   
    x = Dense(512,
              activation='tanh',
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)
    
    x = Dense(48,
              activation=activation,
              kernel_regularizer=regularizers.l2(0.1),
              bias_regularizer=regularizers.l2(0.1),
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(x)

    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
        
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)    
    
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [101]:
run(graph, './models/dense_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
    batch_size=200,
    num_batches=10000)

Running ./models/dense_2017_10_21_16:54:47
Batch 0 Hold-Out Accuracy: 0.340285211802 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.573109030724 Time taken: 30.2s
Batch 2000 Hold-Out Accuracy: 0.571829020977 Time taken: 28.7s
Batch 3000 Hold-Out Accuracy: 0.572524368763 Time taken: 28.2s
Batch 4000 Hold-Out Accuracy: 0.576720535755 Time taken: 28.1s
Batch 5000 Hold-Out Accuracy: 0.575209498405 Time taken: 29.6s
Batch 6000 Hold-Out Accuracy: 0.576876878738 Time taken: 31.1s
Batch 7000 Hold-Out Accuracy: 0.570325255394 Time taken: 30.6s
Batch 8000 Hold-Out Accuracy: 0.576917827129 Time taken: 28.1s
Batch 9000 Hold-Out Accuracy: 0.577545762062 Time taken: 27.6s
DONE TRAINING
FINAL ACCURACY: 0.577545762062
SAVING TO: ./models/dense_2017_10_21_16:54:47/model


## Attempt 2: A number of parallel CovNets

In [113]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        bias_regularizer=regularizers.l2(0.01),    
    )
    
    d1 = Dense(512, **dense_args)(board) #activation='relu')(board) 
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    c1 = Flatten()(Conv2D(1, kernel_size=(4, 4), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))
    c2 = Flatten()(Conv2D(1, kernel_size=(1, 4), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))
    c3 = Flatten()(Conv2D(1, kernel_size=(4, 1), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))
    c4 = Flatten()(Conv2D(1, kernel_size=(5, 5), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))
    c5 = Flatten()(Conv2D(1, kernel_size=(3, 3), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))    
    c6 = Flatten()(Conv2D(1, kernel_size=(2, 2), **conv_args)(rs)) #use_bias=True, activation='relu', input_shape=(7, 6, 1))(rs))

    merged = Concatenate()([d1, c1, c2, c3, c4, c5, c6])
    
    x = Dense(256,  **dense_args)(merged)         
    x = Dense(128,  **dense_args)(x)    
    x = Dense(12,   **dense_args)(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [114]:
sess = run(graph,
           './models/cov2d_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
           batch_size=500,
           num_batches=20000)

Running ./models/cov2d_2017_10_22_11:01:23
Batch 0 Hold-Out Accuracy: 0.481537163258 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.574054598808 Time taken: 29.3s
Batch 2000 Hold-Out Accuracy: 0.58172750473 Time taken: 33.1s
Batch 3000 Hold-Out Accuracy: 0.588079452515 Time taken: 34.4s
Batch 4000 Hold-Out Accuracy: 0.590709269047 Time taken: 33.0s
Batch 5000 Hold-Out Accuracy: 0.590805530548 Time taken: 33.4s
Batch 6000 Hold-Out Accuracy: 0.591366112232 Time taken: 36.8s
Batch 7000 Hold-Out Accuracy: 0.593473851681 Time taken: 37.9s
Batch 8000 Hold-Out Accuracy: 0.593411266804 Time taken: 37.7s
Batch 9000 Hold-Out Accuracy: 0.593921363354 Time taken: 38.2s
Batch 10000 Hold-Out Accuracy: 0.591291546822 Time taken: 33.5s
Batch 11000 Hold-Out Accuracy: 0.593464195728 Time taken: 37.7s
Batch 12000 Hold-Out Accuracy: 0.591301143169 Time taken: 38.2s
Batch 13000 Hold-Out Accuracy: 0.594157159328 Time taken: 37.1s
Batch 14000 Hold-Out Accuracy: 0.595247089863 Time taken: 35.6s
Batch 15000 

## Attempt 3: More complicated covnets

In [140]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')    
    
    dense_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        bias_regularizer=regularizers.l2(0.01),    
    )
    
    d1 = Dense(512, **dense_args)(board) #activation='relu')(board) 
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    rs = Reshape((7, 6, 1), input_shape=(42,))(board)
    
    conv_args = dict(
        use_bias=True,
        activation='relu',
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        kernel_regularizer=regularizers.l2(0.01),
        input_shape=(7, 6, 1),
        padding='valid'
    )
    
    # We use a few parallel covents, that we combine in the end        
    ca = Flatten()(Conv2D(8, kernel_size=(5, 5), **conv_args)(rs))
    #ca2 = Dense(3, **dense_args)(ca1)

    cb = Flatten()(Conv2D(8, kernel_size=(4, 4), **conv_args)(rs))
    #cb2 = Dense(3, **dense_args)(cb1)

    cc = Flatten()(Conv2D(4, kernel_size=(1, 4), **conv_args)(rs))
    #cc2 = Dense(3, **dense_args)(cc1)
    
    cd = Flatten()(Conv2D(4, kernel_size=(4, 1), **conv_args)(rs))
    #cd2 = Dense(3, **dense_args)(cd1)

    merged = Concatenate()([d1, ca, cb, cc, cd])
    
    x = Dense(256,  **dense_args)(merged)         
    x = Dense(128,  **dense_args)(x)    
    x = Dense(12,   **dense_args)(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('evaluation') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [ ]:
sess = run(graph,
           './models/gen1-cov2d_beta_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")),
           batch_size=500,
           num_batches=20000)

Running ./models/gen1-cov2d_beta_2017_10_22_150410
Batch 0 Hold-Out Accuracy: 0.268591493368 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.64998793602 Time taken: 19.7s
Batch 2000 Hold-Out Accuracy: 0.656113445759 Time taken: 21.8s
Batch 3000 Hold-Out Accuracy: 0.656147837639 Time taken: 22.8s
Batch 4000 Hold-Out Accuracy: 0.651226818562 Time taken: 25.7s
Batch 5000 Hold-Out Accuracy: 0.65576928854 Time taken: 25.4s
Batch 6000 Hold-Out Accuracy: 0.650194406509 Time taken: 23.7s
Batch 7000 Hold-Out Accuracy: 0.648577034473 Time taken: 25.7s
Batch 8000 Hold-Out Accuracy: 0.654151916504 Time taken: 23.5s
Batch 9000 Hold-Out Accuracy: 0.652740955353 Time taken: 23.9s
Batch 10000 Hold-Out Accuracy: 0.650848269463 Time taken: 24.9s
Batch 11000 Hold-Out Accuracy: 0.648955583572 Time taken: 27.1s
Batch 12000 Hold-Out Accuracy: 0.650882661343 Time taken: 24.5s
Batch 13000 Hold-Out Accuracy: 0.65370452404 Time taken: 25.0s
